<a href="https://colab.research.google.com/github/glitchheadgit/smilesChessFraud/blob/main/chessFraud_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installs and imports

In [1]:
!pip install transformers datasets bertviz -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 366.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requir

In [14]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm, trange
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader
from datasets import Dataset
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [15]:
ds = pd.read_parquet("part_0.parquet")
chess_moves = [
    "Nf3 Nf6",  # Пример шахматной партии
    "e4 e5",
    "d4 d5",
    "c4 c6",
    "Nf3 d5",
    # Добавьте больше последовательностей ходов
]

labels = [0, 1, 0, 1, 0]  # Могут быть какие-то классы, например 0 и 1 для победы/поражения

# Шаг 2: Создание токенизатора
# Создадим новый токенизатор, который работает с конкретными токенами, представляющими шахматные ходы
chess_tokens = ["Nf3", "Nf6", "e4", "e5", "d4", "d5", "c4", "c6"]  # Список всех возможных ходов
vocab = {token: idx for idx, token in enumerate(chess_tokens, start=1)}
vocab["[PAD]"] = 0  # Добавляем специальный паддинг токен

tokenizer = PreTrainedTokenizerFast(tokenizer_object=None,
                                    unk_token="[UNK]",
                                    pad_token="[PAD]",
                                    eos_token="[EOS]",
                                    cls_token="[CLS]",
                                    mask_token="[MASK]")

tokenizer.add_special_tokens({"additional_special_tokens": chess_tokens})
tokenizer.add_tokens(chess_tokens)

# Токенизация данных
tokenized_inputs = [tokenizer.encode(move, add_special_tokens=True) for move in chess_moves]

# Преобразование в тензоры
inputs = torch.tensor(tokenized_inputs)
labels = torch.tensor(labels)

# Шаг 3: Создание и обучение модели
# Разделение данных на обучающую и тестовую выборки
train_inputs, test_inputs, train_labels, test_labels = train_test_split(inputs, labels, test_size=0.2)

# Создаем модель
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Определение тренера
training_args = TrainingArguments(
    output_dir='./results',          # выходной каталог
    num_train_epochs=3,              # количество эпох
    per_device_train_batch_size=8,   # размер батча для тренировки
    per_device_eval_batch_size=8,    # размер батча для оценки
    warmup_steps=500,                # количество шагов для обучения
    weight_decay=0.01,               # коэффициент затухания
    logging_dir='./logs',            # каталог для логов
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=(train_inputs, train_labels),
    eval_dataset=(test_inputs, test_labels),
)

# Обучение модели
trainer.train()

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.